<a href="https://colab.research.google.com/github/JamesMcGuigan/dlaicourse/blob/master/Exercises/Exercise%204%20-%20Handling%20Complex%20Images/Exercise%204-Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if logs.get('acc') >= DESIRED_ACCURACY:
            self.model.stop_training = True
            print(f"\nReached {round(DESIRED_ACCURACY*100,1)}% accuracy so cancelling training!")

--2020-03-03 07:45:51--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 2a00:1450:4013:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-03-03 07:45:52 (252 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
# This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation=tf.nn.relu, input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),      
    tf.keras.layers.Flatten(),              
    tf.keras.layers.Dense(512, activation=tf.nn.relu),                      
    tf.keras.layers.Dense(1,   activation=tf.nn.sigmoid),                              
])

from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
    metrics=['acc']
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [3]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

# Please use a target_size of 150 X 150.
train_generator = train_datagen.flow_from_directory(
    "/tmp/h-or-s",
    target_size=(150,150),
    batch_size=128,
    class_mode='binary'
)
# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [4]:
# This code block should call model.fit_generator and train for
# a number of epochs.
# model fitting
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8, 
    epochs=15, 
    verbose=1, 
    callbacks=[myCallback()], 
)
# model fitting

Epoch 1/15
8/8 [==============================] - 9s 1s/step - loss: 1.1303 - acc: 0.6187
Epoch 2/15
8/8 [==============================] - 2s 205ms/step - loss: 0.4280 - acc: 0.8016
Epoch 3/15
8/8 [==============================] - 2s 198ms/step - loss: 0.1107 - acc: 0.9734
Epoch 4/15
8/8 [==============================] - 2s 201ms/step - loss: 0.0263 - acc: 0.9969
Epoch 5/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0081 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
8/8 [==============================] - 2s 200ms/step - loss: 0.0077 - acc: 1.0000
